In [2]:
import pandas as pd
import numpy as np
from PIL import Image
from IPython.display import display
import os
import imageio
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms 
import albumentations as A
from albumentations.pytorch import ToTensorV2
import torch
import torch.nn as nn
import torch.optim as optim

In [3]:
!pip install segmentation-models-pytorch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 6.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16446 sha256=cd4d6f548cca8441590bed2c3d086d915600512458b8a507360baa2e2814a35d
  Stored in directory: /root/.cache/pip/wheels/03/3f/e9/911b1bc46869644912bda90a56bcf7b960f20b5187feea3baf
  Created wheel for pretrainedmodels: filename=pretrainedmodels-0.7.4-py3-none-any.whl size=60966 sha256=fcfedaa88d4de8f51169c249f69eb0d36b040c0a5e6ba216c5319be97afa789a
  Stored in directory: /root/.cache/pip/wheels/35/cb/a5/8f534c60142835bfc889f9a482e4a67e0b817032d9c6883b64
Successfully built efficientnet-pytorch pretrainedmodels


In [4]:
import segmentation_models_pytorch as smp
import segmentation_models_pytorch.utils.losses

In [5]:
folders = ['dataA','dataB','dataC','dataD','dataE']
image_folder = 'CameraRGB'
mask_folder = 'CameraSeg'

In [6]:
data = []
base = '../input/lyft-udacity-challenge/'
for folder in folders:
    image_path = os.path.join(base,folder+"/"+folder, image_folder)
    mask_path = os.path.join(base,folder+"/"+folder, mask_folder)
    
    image_files = os.listdir(image_path)
    mask_files = os.listdir(mask_path)
    
    for image_file, mask_file in zip(image_files, mask_files):
        image_full_path = os.path.join(image_path, image_file)
        mask_full_path = os.path.join(mask_path, mask_file)
        
        data.append({'id': image_file.split('.')[0], 'imagepath': image_full_path, 'maskpath': mask_full_path})

df = pd.DataFrame(data)

In [7]:
class SDCDataSet(Dataset):
    def __init__(self,imgData):
        self.train = train
        self.imgData = imgData
        self.transform = A.Compose([
                            A.Resize(width=576, height=576),
                            A.HorizontalFlip(p=0.2),
                            A.RandomBrightnessContrast(p=0.2),
                            ToTensorV2(transpose_mask=True)
                        ])
    def __len__(self):
        return len(self.imgData)
    def __getitem__(self,idx):
        img = imageio.v3.imread(self.imgData.iloc[idx].imagepath)
        mask = imageio.v3.imread(self.imgData.iloc[idx].maskpath,mode="L")
        img = img.astype("float32")/255.0
        mask = mask.astype("float32") 
        transformed = self.transform(image=img, mask=mask)
        return transformed['image'],transformed['mask']  

In [8]:
train,test = train_test_split(df,test_size=0.2)

In [9]:
train_ds = SDCDataSet(train)
valid_ds = SDCDataSet(test)

In [10]:
img,mask = train_ds[0]
print(f"img size {img.shape} and mask shape {mask.shape}")

img size torch.Size([3, 576, 576]) and mask shape torch.Size([576, 576])


In [11]:
train_dl = DataLoader(train_ds,batch_size=8)
valid_dl = DataLoader(valid_ds,batch_size=8)

In [12]:
device = torch.device('cuda:0')


In [13]:
model = smp.Unet(in_channels=3,classes=2)

Downloading: "https://download.pytorch.org/models/resnet34-333f7ec4.pth" to /root/.cache/torch/hub/checkpoints/resnet34-333f7ec4.pth
100%|██████████| 83.3M/83.3M [00:01<00:00, 71.1MB/s]


In [14]:
model = model.to(device)

In [15]:
loss = smp.utils.losses.DiceLoss()
metrics = [
    smp.utils.metrics.IoU(threshold=0.5),
]

optimizer = torch.optim.Adam([ 
    dict(params=model.parameters(), lr=0.0001),
])

scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=0.01, steps_per_epoch=len(train_dl), epochs=10)

In [ ]:
epochs = 1
